In [1]:
from math import sqrt
import copy
import  traceback
import shutil
import random

import numpy as np  # linear algebra
import pydicom
import os
import matplotlib.pyplot as plt
import cv2
from pydicom.uid import UID
from PIL import Image
from tqdm import tqdm
import openpyxl

In [2]:
def load_scan(path):
    slices = []
    for s in os.listdir(path):
        if not s.endswith('.dcm'):
            continue
        sl = pydicom.dcmread(os.path.join(path, s))
        try:
            sl_p = sl.pixel_array
        except AttributeError:
            traceback.print_exc()
            print(f'\tDelete {os.path.join(path, s)}')
            os.remove(os.path.join(path, s))
        else:
            slices.append(sl)
    slices.sort(key=lambda x: float(x.InstanceNumber))
    return slices, slices[0].SliceThickness

In [3]:
lower_b, upper_b = -100, 500

In [ ]:
root = '/nfs3-p1/zsxm/dataset/2021-10-19-pau'
not_2_list = []
for patient in sorted(os.listdir(root)):
    patient = os.path.join(root, patient)
    if os.path.isfile(patient):
        continue
    if len(os.listdir(patient)) > 2:
        not_2_list.append(patient)
        continue
    ct, ct_thickness = load_scan(os.path.join(patient, '1'))
    cta, cta_thickness = load_scan(os.path.join(patient, '2'))
    if ct_thickness % cta_thickness != 0:
        print(patient, ct_thickness, cta_thickness)

In [ ]:
for patient in not_2_list:
    print(patient)
print(len(not_2_list))

In [ ]:
root = '/nfs3-p1/zsxm/dataset/2021-10-19-imh'
wb = openpyxl.load_workbook(os.path.join(root, 'label.xlsx'))
sheet = wb['label']
for patient in sorted(os.listdir(root)):
    name = patient.split('-')[0]
    patient = os.path.join(root, patient)
    if os.path.isfile(patient):
        continue
    for row in sheet.iter_rows():
        if row[0].value == name:
            ct_label = row[3].value.replace('\n', ' ').replace(' ', '-').upper().replace('S', '').replace('J', '').replace('G', '')
            cta_label = row[4].value.replace('\n', ' ').replace(' ', '-').upper().replace('S', '').replace('J', '').replace('G', '')
            break
    print(name, ct_label, cta_label)
    ctl, ctal = ct_label.split('-'), cta_label.split('-')
    assert len(ctl) == len(ctal), name
    ct_start, ct_end, cta_start, cta_end = 10000, -1, 10000, -1
    ct_min_i, ct_max_i, cta_min_i, cta_max_i = -1, -1, -1, -1
    for i in range(len(ctl)):
        if int(ctl[i])-1 < ct_start:
            ct_start = int(ctl[i])-1
            ct_min_i = i
        if int(ctl[i])-1 > ct_end:
            ct_end = int(ctl[i])-1
            ct_max_i = i
        if int(ctal[i])-1 < cta_start:
            cta_start = int(ctal[i])-1
            cta_min_i = i
        if int(ctal[i])-1 > cta_end:
            cta_end = int(ctal[i])-1
            cta_max_i = i
        assert ct_min_i == cta_min_i, f'{len(ct)}  {len(cta)}'
        assert ct_max_i == cta_max_i, f'{len(ct)}  {len(cta)}'
    m, n = ct_end - ct_start, cta_end - cta_start
    assert m>0 and n>0
    print(m, n)

In [29]:
def combine_image0(root):    
    wb = openpyxl.load_workbook(os.path.join(root, 'label.xlsx'))
    sheet = wb['label']
    size_not_match = set()
    for patient in sorted(os.listdir(root)):
        name = patient.split('-')[0]
        folder_name = patient
        patient = os.path.join(root, patient)
        if os.path.isfile(patient):
            continue
        ct, ct_thickness = load_scan(os.path.join(patient, '1'))
        cta, cta_thickness = load_scan(os.path.join(patient, '2'))
        for row in sheet.iter_rows():
            if row[0].value == name:
                ct_label = row[3].value.replace('\n', ' ').replace(' ', '-').upper().replace('S', '').replace('J', '').replace('G', '')
                cta_label = row[4].value.replace('\n', ' ').replace(' ', '-').upper().replace('S', '').replace('J', '').replace('G', '')
                break
        print(name, ct_label, cta_label)
        ctl, ctal = ct_label.split('-'), cta_label.split('-')
        assert len(ctl) == len(ctal), name
        ct_cta_table = [-len(cta)] * len(ct)
        if ct_thickness % cta_thickness == 0:
            times = int(ct_thickness // cta_thickness)
            ct_pivot, cta_pivot = int(ctl[0])-1, int(ctal[0])-1
            for i in range(len(ct)):
                ct_cta_table[i] = min(max(cta_pivot + (i - ct_pivot) * times, 0), len(cta)-1)
        else:
            ct_start, ct_end, cta_start, cta_end = int(ctl[0])-1, int(ctl[1])-1, int(ctal[0])-1, int(ctal[1])-1
            if ct_start > ct_end:
                ct_start, ct_end, cta_start, cta_end = ct_end, ct_start, cta_end, cta_start
            m, n = ct_end - ct_start, cta_end - cta_start
            ot, q = [], 0
            for p in range(m):
                min_dis = abs(q/n-p/m)
                while q < n:
                    q += 1
                    if abs(q/n-p/m) < min_dis:
                        min_dis = abs(q/n-p/m)
                    else:
                        q -= 1
                        ot.append(q)
                        break
            print(ot)
            assert len(ot) == m, name
            for i in range(len(ct)):
                ct_cta_table[i] = min(max(cta_start + (i-ct_start)//m*n + ot[(i-ct_start)%m], 0), len(cta)-1)
        #print(ct_cta_table)
        assert min(ct_cta_table) >=0 and max(ct_cta_table) < len(cta), f'{name}:{min(ct_cta_table)},{max(ct_cta_table)},{len(cta)}'
        image_path = os.path.join(patient, f'combine_images_{lower_b}_{upper_b}')
        if os.path.exists(image_path):
             shutil.rmtree(image_path)
        os.mkdir(image_path)
        for i in range(len(ct)):
            ct_img = ct[i].pixel_array.astype(np.int16)
            intercept = ct[i].RescaleIntercept
            slope = ct[i].RescaleSlope
            if slope != 1:
                ct_img = (slope * ct_img.astype(np.float64)).astype(np.int16)
            ct_img += np.int16(intercept)
            ct_img = np.clip(ct_img, lower_b, upper_b)
            ct_img = ((ct_img-lower_b)/(upper_b-lower_b)*255).astype(np.uint8)

            cta_img = cta[ct_cta_table[i]].pixel_array.astype(np.int16)
            intercept = cta[ct_cta_table[i]].RescaleIntercept
            slope = cta[ct_cta_table[i]].RescaleSlope
            if slope != 1:
                cta_img = (slope * cta_img.astype(np.float64)).astype(np.int16)
            cta_img += np.int16(intercept)
            cta_img = np.clip(cta_img, lower_b, upper_b)
            cta_img = ((cta_img-lower_b)/(upper_b-lower_b)*255).astype(np.uint8)

            if ct_img.shape[0] < cta_img.shape[0]:
                crop_start = int(round((cta_img.shape[0]-ct_img.shape[0])/2.))
                cta_img = cta_img[crop_start: crop_start+ct_img.shape[0]]
            if ct_img.shape[1] < cta_img.shape[1]:
                crop_start = int(round((cta_img.shape[1]-ct_img.shape[1])/2.))
                cta_img = cta_img[:, crop_start: crop_start+ct_img.shape[1]]
            if ct_img.shape[0] > cta_img.shape[0] or ct_img.shape[1] > cta_img.shape[1]:
                size_not_match.add(patient)
                break

            img = np.concatenate((ct_img, cta_img), axis=1)
            img = Image.fromarray(img)
            img.save(os.path.join(image_path, f'{folder_name}_{i:04d}_{ct_cta_table[i]:04d}.png'))
            
combine_image0('/nfs3-p1/zsxm/dataset/2021-10-19-imh')
combine_image0('/nfs3-p1/zsxm/dataset/2021-10-19-pau')

chencaixi 11-53 53-253
[2, 7, 12, 17, 22, 27, 32, 37, 42, 47, 52, 57, 62, 67, 72, 77, 82, 87, 92, 97, 102, 107, 112, 117, 122, 127, 132, 137, 142, 147, 152, 157, 162, 167, 172, 177, 182, 187, 192, 197, 202, 207, 212, 217, 222, 227, 232, 237, 242, 247, 252, 257, 262, 267, 272, 277, 282, 287, 292, 297, 302, 307, 312, 317, 322, 327, 332, 337, 342, 347, 352, 357, 362, 367, 372, 377, 382, 387, 392, 397, 402, 407, 412, 417, 422, 427, 432, 437, 442, 447, 452, 457, 462, 467, 472, 477, 482, 487, 492, 497, 502, 507, 512, 517, 522, 527, 532, 537, 542, 547, 552, 557, 562, 567, 572, 577, 582, 587, 592, 597]
chenchanglu 142-168 142-168
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94,

heazi 30-16 51-27
[0, 2, 3, 5, 7, 9, 10, 12, 14, 15, 17, 19, 21, 22]
[0, 2, 4, 5, 7, 9, 11, 12, 14, 16, 17, 19, 21, 23, 24, 26, 28, 29, 31, 33, 35, 36, 38, 40, 41, 43, 45, 47, 48, 50, 52, 53, 55, 57, 59, 60, 62, 64, 65, 67, 69, 71, 72, 74, 76, 77, 79, 81, 83, 84, 86, 88, 89, 91, 93, 95, 96, 98, 100, 101, 103, 105, 107, 108, 110, 112, 113, 115, 117, 119, 120, 122, 124, 125, 127, 129, 131, 132, 134, 136, 137, 139, 141, 143, 144, 146, 148, 149, 151, 153, 155, 156, 158, 160, 161, 163, 165, 167, 168, 170, 172, 173, 175, 177, 179, 180, 182, 184, 185, 187, 189, 191, 192, 194, 196, 197, 197, 197]
hezhengmin 24-74 48-147
[1, 3, 5, 7, 9, 11, 13, 15, 17, 19, 21, 23, 25, 27, 29, 31, 33, 35, 37, 39, 41, 43, 45, 47, 49, 51, 53, 55, 57, 59, 61, 63, 65, 67, 69, 71, 73, 75, 77, 79, 81, 83, 85, 87, 89, 91, 93, 95, 97, 99, 101, 103, 105, 107, 109, 111, 113, 115, 117, 119, 121, 123, 125, 127, 129, 131, 133, 135, 137, 139, 141, 143, 145, 147, 149, 151, 153, 155, 157, 159, 161, 163, 165, 167, 169, 171, 173,

jiatiandeng 55-32-142 173-100-433
[10, 13, 16, 19, 22, 25, 28, 31, 34, 37, 40, 43, 46, 49, 52, 55, 58, 61, 64, 67, 70, 73, 76, 79, 82, 85, 88, 91, 94, 97, 100, 103, 106, 109, 112, 115, 118, 121, 124, 127, 130, 133, 136, 139, 142, 145, 148, 151, 154, 157, 160, 163, 166, 169, 172, 175, 178, 181, 184, 187, 190, 193, 196, 199, 202, 205, 208, 211, 214, 217, 220, 223, 226, 229, 232, 235, 238, 241, 244, 247, 250, 253, 256, 259, 262, 265, 268, 271, 274, 277, 280, 283, 286, 289, 292, 295, 298, 301, 304, 307, 310, 313, 316, 319, 322, 325, 328, 331, 334, 337, 340, 343, 346, 349, 352, 355, 358, 361, 364, 367, 370, 373, 376, 379, 382, 385, 388, 391, 394, 397, 400, 403, 406, 409, 412, 415, 418, 421, 424, 427, 430, 433, 436, 439, 442, 445, 448, 451, 454, 457, 460, 463, 466, 469, 472, 475, 478, 481, 484, 487, 490, 493, 496, 499, 502, 505, 508, 511, 514, 517, 520, 523, 526, 529, 532, 535, 538, 541, 544, 547, 548, 548, 548]
jinguoqing 15-24-15-76 18-27-15-79
[3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15,

lisong 30-158 89-473
[1, 4, 7, 10, 13, 16, 19, 22, 25, 28, 31, 34, 37, 40, 43, 46, 49, 52, 55, 58, 61, 64, 67, 70, 73, 76, 79, 82, 85, 88, 91, 94, 97, 100, 103, 106, 109, 112, 115, 118, 121, 124, 127, 130, 133, 136, 139, 142, 145, 148, 151, 154, 157, 160, 163, 166, 169, 172, 175, 178, 181, 184, 187, 190, 193, 196, 199, 202, 205, 208, 211, 214, 217, 220, 223, 226, 229, 232, 235, 238, 241, 244, 247, 250, 253, 256, 259, 262, 265, 268, 271, 274, 277, 280, 283, 286, 289, 292, 295, 298, 301, 304, 307, 310, 313, 316, 319, 322, 325, 328, 331, 334, 337, 340, 343, 346, 349, 352, 355, 358, 361, 364, 367, 370, 373, 376, 379, 382, 385, 388, 391, 394, 397, 400, 403, 406, 409, 412, 415, 418, 421, 424, 427, 430, 433, 436, 439, 442, 445, 448, 451, 454, 457, 460, 463, 466, 469, 472, 475, 478, 481, 484, 487, 490, 493, 496, 499, 502, 505, 508, 511, 514, 517, 520, 523, 526, 529, 532, 535, 538, 541, 544, 547, 550, 553, 556, 559, 562, 565, 568, 571, 574, 577, 580, 583, 586, 589, 592, 595, 598, 601, 604, 607,

qichunhong 27-35 80-104
[1, 4, 7, 10, 13, 16, 19, 22, 25, 28, 31, 34, 37, 40, 43, 46, 49, 52, 55, 58, 61, 64, 67, 70, 73, 76, 79, 82, 85, 88, 91, 94, 97, 100, 103, 106, 109, 112, 115, 118, 121, 124, 127, 130, 133, 136, 139, 142, 145, 148, 151, 154, 157, 160, 163, 166, 169, 172, 175, 178, 181, 184, 187, 190, 193, 196, 199, 202, 205, 208, 211, 214, 217, 220, 223, 226, 229, 232, 235, 238, 241, 244, 247, 250, 253, 256, 259, 262, 265, 268, 271, 274, 277, 280, 283, 286, 289, 292, 295, 298, 301, 304, 307, 310, 313, 316, 319, 322, 325, 328, 331, 334, 337, 340, 343, 346, 349, 352, 355, 358, 361, 364, 367, 370, 373, 376, 379, 382, 385, 388, 391, 394, 397, 400, 403, 406, 409, 412, 415, 418, 421, 424, 427, 430, 433, 436, 439, 442, 445, 448, 451, 454, 457, 460, 463, 466, 469, 472, 475, 478, 481, 484, 487, 490, 493, 496, 499, 502, 505, 508, 511, 514, 517, 520, 523, 526, 529, 532, 535, 538, 541, 544, 547, 550, 553, 556, 559, 562, 565, 568, 571, 574, 577, 580, 583, 586, 589, 592, 595, 598, 601, 604, 6

...

	Delete /nfs3-p1/zsxm/dataset/2021-10-19-imh/tianjianjun-S-40-65-J-40-125/2/2-2.5mm-0105.dcm
tianjianjun 40-65-40-125 40-65-40-125
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195

wuxisheng 31-23-27 29-21-25
[0, 0, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132]
wuzonghe 11-62 15-100
[0, 2, 3, 5, 7, 8, 10, 12, 13, 15, 17, 18, 20, 22, 23, 25, 27, 28, 30, 32, 33, 35, 37, 38, 40, 42, 43, 45, 47, 48, 50, 52, 53, 55, 57, 58, 60, 62, 63, 65, 67, 68, 70, 72, 73, 75, 77, 78, 80, 82, 83]
[0, 0, 1, 2, 4, 6, 7, 9, 11, 12, 14, 16, 17, 19, 21, 22, 24, 26, 27, 29, 31, 32, 34, 36, 37, 39, 41, 42, 44, 46, 47, 49, 51, 52, 54, 56, 57, 59, 61, 62, 64, 66, 67, 69, 71, 72, 74, 76, 77, 

xuzhengcheng 18-58 53-173
[1, 4, 7, 10, 13, 16, 19, 22, 25, 28, 31, 34, 37, 40, 43, 46, 49, 52, 55, 58, 61, 64, 67, 70, 73, 76, 79, 82, 85, 88, 91, 94, 97, 100, 103, 106, 109, 112, 115, 118, 121, 124, 127, 130, 133, 136, 139, 142, 145, 148, 151, 154, 157, 160, 163, 166, 169, 172, 175, 178, 181, 184, 187, 190, 193, 196, 199, 202, 205, 208, 211, 214, 217, 220, 223, 226, 229, 232, 235, 238, 241, 244, 247, 250, 253, 256, 259, 262, 265, 268, 271, 274, 277, 280, 283, 286, 289, 292, 295, 298, 301, 304, 307, 310, 313, 316, 319, 322, 325, 328, 331, 334, 337, 340, 343, 346, 349, 352, 355, 358, 361, 364, 367, 370, 373, 376, 379, 382, 385, 388, 391, 394, 397, 400, 403, 406, 409, 412, 415, 418, 421, 424, 427, 430, 433, 436, 439, 442, 445, 448, 451, 454, 457, 460, 463, 466, 469, 472, 475, 478, 481, 484, 487, 490, 493, 496, 499, 502, 505, 508, 511, 514, 517, 520, 523, 526, 529, 532, 535, 538, 541, 544, 547, 550, 553, 556, 559, 562, 565, 568, 571, 574, 577, 580, 583, 586, 589, 592, 595, 598, 601, 604,

zhanglvhao 24-41 116-202
[0, 5, 10, 15, 20, 25, 30, 35, 40, 45, 50, 55, 60, 65, 70, 75, 80, 85, 90, 95, 100, 105, 110, 115, 120, 125, 130, 135, 140, 145, 150, 155, 160, 165, 170, 175, 180, 185, 190, 195, 200, 205, 210, 215, 220, 225, 230, 235, 240, 245, 250, 255, 260, 265, 270, 275, 280, 285, 290, 295, 300, 305, 310, 315, 320, 325, 330, 335, 340, 345, 350, 355, 360, 365, 370, 375, 380, 385, 390, 395, 400, 405, 410, 415, 420, 425, 430, 435, 440, 445, 450, 455, 460, 465, 470, 475, 480, 485, 490, 495, 500, 505, 510, 515, 520, 525, 530, 535, 540, 545, 550, 555, 560, 565, 570, 575, 580, 585, 590]
zhangrijun 23-66 25-68
[2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 9

zhuxuequn 97-104 92-99
[0, 0, 0, 0, 0, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203]
zhuyucai 19-29 20-30
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10

In [50]:
def combine_image(root):    
    wb = openpyxl.load_workbook(os.path.join(root, 'label.xlsx'))
    sheet = wb['label']
    size_not_match = set()
    for patient in sorted(os.listdir(root)):
        name = patient.split('-')[0]
        folder_name = patient
        patient = os.path.join(root, patient)
        if os.path.isfile(patient):
            continue
        ct, ct_thickness = load_scan(os.path.join(patient, '1'))
        cta, cta_thickness = load_scan(os.path.join(patient, '2'))
        for row in sheet.iter_rows():
            if row[0].value == name:
                ct_label = row[3].value.replace('\n', ' ').replace(' ', '-').upper().replace('S', '').replace('J', '').replace('G', '')
                cta_label = row[4].value.replace('\n', ' ').replace(' ', '-').upper().replace('S', '').replace('J', '').replace('G', '')
                break
        print(name, ct_label, cta_label)
        ctl, ctal = ct_label.split('-'), cta_label.split('-')
        assert len(ctl) == len(ctal), name
        ct_cta_table = [-len(cta)] * len(ct)
        ct_start, ct_end, cta_start, cta_end = len(ct), -1, len(cta), -1
        ct_min_i, ct_max_i, cta_min_i, cta_max_i = -1, -1, -1, -1
        for i in range(len(ctl)):
            if int(ctl[i])-1 < ct_start:
                ct_start = int(ctl[i])-1
                ct_min_i = i
            if int(ctl[i])-1 > ct_end:
                ct_end = int(ctl[i])-1
                ct_max_i = i
            if int(ctal[i])-1 < cta_start:
                cta_start = int(ctal[i])-1
                cta_min_i = i
            if int(ctal[i])-1 > cta_end:
                cta_end = int(ctal[i])-1
                cta_max_i = i
            assert ct_min_i == cta_min_i and ct_max_i == cta_max_i
        m, n = ct_end - ct_start, cta_end - cta_start
        assert m > 0 and n > 0
        ot, q = [], 0
        for p in range(m):
            min_dis = abs(q/n-p/m)
            while q < n:
                q += 1
                if abs(q/n-p/m) < min_dis:
                    min_dis = abs(q/n-p/m)
                else:
                    q -= 1
                    ot.append(q)
                    break
        #print(ot)
        assert len(ot) == m, name
        for i in range(len(ct)):
            ct_cta_table[i] = min(max(cta_start + (i-ct_start)//m*n + ot[(i-ct_start)%m], 0), len(cta)-1)
        #print(ct_cta_table)
        assert min(ct_cta_table) >=0 and max(ct_cta_table) < len(cta), f'{name}:{min(ct_cta_table)},{max(ct_cta_table)},{len(cta)}'
        image_path = os.path.join(patient, f'combine_images_{lower_b}_{upper_b}')
        if os.path.exists(image_path):
             shutil.rmtree(image_path)
        os.mkdir(image_path)
        for i in range(len(ct)):
            ct_img = ct[i].pixel_array.astype(np.int16)
            intercept = ct[i].RescaleIntercept
            slope = ct[i].RescaleSlope
            if slope != 1:
                ct_img = (slope * ct_img.astype(np.float64)).astype(np.int16)
            ct_img += np.int16(intercept)
            ct_img = np.clip(ct_img, lower_b, upper_b)
            ct_img = ((ct_img-lower_b)/(upper_b-lower_b)*255).astype(np.uint8)

            cta_img = cta[ct_cta_table[i]].pixel_array.astype(np.int16)
            intercept = cta[ct_cta_table[i]].RescaleIntercept
            slope = cta[ct_cta_table[i]].RescaleSlope
            if slope != 1:
                cta_img = (slope * cta_img.astype(np.float64)).astype(np.int16)
            cta_img += np.int16(intercept)
            cta_img = np.clip(cta_img, lower_b, upper_b)
            cta_img = ((cta_img-lower_b)/(upper_b-lower_b)*255).astype(np.uint8)

            if ct_img.shape[0] < cta_img.shape[0]:
                crop_start = int(round((cta_img.shape[0]-ct_img.shape[0])/2.))
                cta_img = cta_img[crop_start: crop_start+ct_img.shape[0]]
            if ct_img.shape[1] < cta_img.shape[1]:
                crop_start = int(round((cta_img.shape[1]-ct_img.shape[1])/2.))
                cta_img = cta_img[:, crop_start: crop_start+ct_img.shape[1]]
            if ct_img.shape[0] > cta_img.shape[0] or ct_img.shape[1] > cta_img.shape[1]:
                size_not_match.add(patient)
                break

            img = np.concatenate((ct_img, cta_img), axis=1)
            img = Image.fromarray(img)
            img.save(os.path.join(image_path, f'{folder_name}_{i+1:04d}_{ct_cta_table[i]+1:04d}.png'))
            
combine_image('/nfs3-p1/zsxm/dataset/2021-10-19-imh')
combine_image('/nfs3-p1/zsxm/dataset/2021-10-19-pau')

In [49]:
def move_together(root, dist):
    os.makedirs(dist, exist_ok=True)
    for patient in sorted(os.listdir(root)):
        if os.path.isfile(os.path.join(root, patient)):
            continue
        print(f'Processing {patient}')
        img_path = os.path.join(root, patient, f'combine_images_{lower_b}_{upper_b}')
        shutil.copytree(img_path, os.path.join(dist, patient))

move_together('/nfs3-p1/zsxm/dataset/2021-10-19-imh', '/nfs3-p1/zsxm/dataset/aorta_combine/imh')
move_together('/nfs3-p1/zsxm/dataset/2021-10-19-pau', '/nfs3-p1/zsxm/dataset/aorta_combine/pau')

Processing chencaixi-J-16-53
Processing chenchanglu-J-142-168
Processing chengchunxiao-S-39-57-J-39-93
Processing chenyiqing-J-69-117
Processing chenyouli-J-34-93
Processing chenzhiqing-J-19-55
Processing dengxiuzhu-S-27-47-J-27-86
Processing fangmengzhen-J-20-73
Processing guanzaihua-J-25-61
Processing heazi-S-16-30
Processing hezhengmin-J-24-74
Processing hongshunchang-J-26-89
Processing huangbaichao-J-31-105
Processing hucailu-J-32-55
Processing huyaxiang-J-40-89
Processing huyingfu-S-31-39-J-31-105
Processing jiangbihua-S-21-30-J-21-55
Processing jiangjianping-J-17-63
Processing jiangqianfeng-J-35-98
Processing jiatiandeng-S-41-55-J-41-142
Processing jinguoqing-S-15-24-J-15-76
Processing kebaoming-S-29-40
Processing laidaishan-J-24-47
Processing lichengqian-J-35-100
Processing liguihua-S-22-24
Processing limingyu-J-27-55
Processing lindepei-J-136-158
Processing lingsulan-J-46-107
Processing linjian-J-34-105
Processing lisanliang-J-37-101
Processing lisong-J-39-158
Processing luae-J

In [53]:
positive_folders = ['/nfs3-p2/zsxm/dataset/2021-9-8/','/nfs3-p2/zsxm/dataset/2021-9-13/','/nfs3-p2/zsxm/dataset/2021-9-19/','/nfs3-p2/zsxm/dataset/2021-9-28/']
patient_list = []
for folder in positive_folders:
    for patient in sorted(os.listdir(folder)):
        patient = os.path.join(folder, patient)
        if os.path.isdir(patient):
            patient_list.append(patient)
            
            
            
selected_list = ['baochunsheng-J-20-82', 'caiweiguang-J-Im35-152', 'chenbo-S-Im24-45-J-Im24-97', 'chenfujun-J-Im43-100', 'chengshizheng-J-Im30-145', 'chengyonghui-J-17-81', 'chenjiandong-S-19-28-J-19-91', 'chenping-J-Im23-88', 'chensiqi-S-17-28-J-17-69', 'chugentang-J-Im19-80', 'fangchunfeng-J-16-48', 'fanghongwei-J-22-71', 'guanxujun-S-Im25-35-J-Im25-87', 'guoheying-J-Im32-127', 'guquankang-J-Im18-69', 'huangdebing-S-Im40-48-J-Im33-138', 'huanghai-S-Im19-29-J-Im19-80', 'huangsuyue-J-Im21-41', 'huangxihong-J-Im27-136', 'huangyong-J-Im18-127', 'huangyongliang-J-Im16-81', 'jiadeen-J-Im34-148', 'jihongke-S-Im20-24-J-Im20-86', 'laiguizhen-J-Im38-121', 'laixuexiang-J-Im33-148', 'lanshaomei-S-Im20-31-J-Im20-79', 'leixiaoying-S-Im18-26-J-18-71', 'lihuanhuan-S-Im20-28-J-Im20-59', 'liubihai-S-23-26-J-23-93', 'liyehua-S-Im30-47-J-Im30-131', 'louyidong-S-Im38-58-J-Im38-133', 'luojun-J-Im22-81', 'maliwei-S-Im30-48-J-Im30-101', 'pengzhengjiang-J-Im38-160', 'qumin-S-Im22-35-J-Im22-28', 'ruweiping-S-Im52-63-J-Im52-94', 'shaoyefeng-S-Im34-43-J-Im34-48', 'shenglina-J-Im21-65', 'shenqi-J-Im24-72', 'tangabiao-S-Im41-49-J-Im41-138', 'wangjiangwei-J-Im37-143', 'wangxiaofu-S-19-29-J-19-65', 'wangyonghui-S-Im18-27-J-Im18-84', 'wangyuefeng-S-Im34-38-J-Im34-144', 'wangzhengjin-J-16-82', 'wenyongguo-J-Im23-121', 'wuyueming-J-Im36-138', 'xiaminsong-S-Im40-51-J-Im40-139', 'xiashubiao-J-24-88', 'xuhangying-J-25-63', 'xuyaofeng-J-Im24-119', 'xuzhaofang-S-21-26-J-21-86', 'yangmin-J-Im37-150', 'yangxuehua-J-Im23-79', 'yangzhanxiang-J-Im31-145', 'yepeng-J-Im26-125', 'yexianchang-S-17-30-J-17-74', 'yexiyou-J-Im35-139', 'yujiada-J-Im21-48', 'zhangguangming-J-21-138', 'zhanghaitao-S-Im25-49-J-Im25-145', 'zhangwei-S-20-26-J-20-61', 'zhangyunqin-S-Im14-25-J-Im14-64', 'zhaowenxian-J-Im43-111', 'zhengguozheng-J-18-81', 'zhengjiyou-J-Im28-136', 'zhoufeng-S-Im35-40-J-Im35-147', 'zhouliang-J-Im47-123', 'zhuseng-S-Im24-31-J-Im24-105', 'zhuxutao-J-Im23-78', 'zhuyongfu-J-Im22-80', 'zhuyuejin-J-Im21-72', 'zongminghui-S-Im18-25-J-Im18-81']+['chenggang-J-Im18-81', 'chenjufa-J-16-66', 'chenjuli-J-Im18-49', 'chenjun-J-Im33-137', 'chenlili-S-Im18-24-J-Im18-74', 'chiyanfei-J-Im20-83', 'daizuokou-J-Im19-74', 'ganxiaobin-J-23-51', 'guojianfu-J-Im32-100', 'hongzhimin-S-Im30-39-J-30-144', 'huanglijun-J-24-98', 'huleijun-J-Im25-141', 'jilixian-J-Im34-108', 'jinchongfei-J-22-92', 'lijianming-J-Im32-124', 'lingenqiang-S-Im30-39', 'linjiaxiang-J-21-72', 'liuyunfei-J-Im40-95', 'loulinhua-S-Im27-30', 'lufeng-J-Im28-103', 'luzhiping-S-Im25-35-J-Im25-54', 'panzhangsong-J-Im26-91', 'qianfuying-J-Im20-87', 'qiaowei-S-Im18-30-J-Im18-82', 'shanghongjun-J-22-84', 'shaoxiulan-J-Im21-25', 'shengwenping-S-Im37-46-J-Im37-116', 'shenliqiang-J-Im29-137', 'shenxuefu-S-Im33-40-J-Im33-101', 'sishouzhong-J-Im25-137', 'wangqing-J-35-102', 'wutonggen-S-Im22-29-J-Im22-82', 'wuwanglong-J-Im31-125', 'wuxiangyang-S-Im18-25', 'xiafangzhou-S-18-25-J-18-80', 'xuhong-S-Im23-38-J-Im23-40', 'xushichao-J-Im35-145', 'yangbingshui-S-Im25-33-J-Im25-56', 'yanjuanfeng-J-Im30-32', 'yaocaiming-J-Im26-88', 'yaojianmin-S-Im20-29-J-Im20-86', 'yesanghua-J-Im41-138', 'yewenyi-J-Im36-130', 'yingguoliang-J-Im55-130', 'yingjianquan-S-Im12-14-J-Im12-36', 'yingmeiqi-S-Im22-25-J-Im16-36', 'yintianxing-J-Im24-38', 'yuanlinyue-J-Im27-120', 'yuguiying-S-16-22-J-16-77', 'yuhongliang-S-Im26-37-J-Im26-105', 'yujiazhen-S-Im19-27-J-Im19-77', 'zhangboqian-S-Im22-32-J-Im22-88', 'zhangfuyang-S-Im15-20-J-Im15-87', 'zhangjian-J-26-89', 'zhanglimin-J-Im40-149', 'zhongxuefang-S-Im19-34-J-Im19-72', 'zhoubozhong-J-Im29-131', 'zhoumingfang-J-Im36-41', 'zhousuhua-J-Im25-96', 'zhuhancheng-S-Im27-36-J-Im27-86']
processed_list = []
for patient in patient_list:
    if patient.split('/')[-1] in selected_list:
        print(patient)
        processed_list.append(patient)
        

/nfs3-p2/zsxm/dataset/2021-9-8/chengshizheng-J-Im30-145
/nfs3-p2/zsxm/dataset/2021-9-8/chenjuli-J-Im18-49
/nfs3-p2/zsxm/dataset/2021-9-8/guanxujun-S-Im25-35-J-Im25-87
/nfs3-p2/zsxm/dataset/2021-9-8/guoheying-J-Im32-127
/nfs3-p2/zsxm/dataset/2021-9-8/hongzhimin-S-Im30-39-J-30-144
/nfs3-p2/zsxm/dataset/2021-9-8/huangxihong-J-Im27-136
/nfs3-p2/zsxm/dataset/2021-9-8/huangyongliang-J-Im16-81
/nfs3-p2/zsxm/dataset/2021-9-8/huleijun-J-Im25-141
/nfs3-p2/zsxm/dataset/2021-9-8/jiadeen-J-Im34-148
/nfs3-p2/zsxm/dataset/2021-9-8/jihongke-S-Im20-24-J-Im20-86
/nfs3-p2/zsxm/dataset/2021-9-8/laiguizhen-J-Im38-121
/nfs3-p2/zsxm/dataset/2021-9-8/liuyunfei-J-Im40-95
/nfs3-p2/zsxm/dataset/2021-9-8/liyehua-S-Im30-47-J-Im30-131
/nfs3-p2/zsxm/dataset/2021-9-8/louyidong-S-Im38-58-J-Im38-133
/nfs3-p2/zsxm/dataset/2021-9-8/luzhiping-S-Im25-35-J-Im25-54
/nfs3-p2/zsxm/dataset/2021-9-8/qiaowei-S-Im18-30-J-Im18-82
/nfs3-p2/zsxm/dataset/2021-9-8/qumin-S-Im22-35-J-Im22-28
/nfs3-p2/zsxm/dataset/2021-9-8/shaoxiulan-J-Im

In [59]:
ksize = 1
print(isinstance(ksize, int))

True
